In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
import time
import itertools

In [ ]:
def tic():
    global __start_interval 
    __start_interval = time.perf_counter()
def toc():
    global __start_interval
    duration = time.perf_counter() - __start_interval
    print(f"Duration = {duration:.2f}")
def toc1():
    global __start_interval
    duration = time.perf_counter() - __start_interval
    return duration

In [ ]:
exist = os.path.exists("Downloaded_TCGA")
if not exist: os.makedirs("Downloaded_TCGA")

In [ ]:
# download data with arguments corresponding to output file name
!Rscript download_data.r Downloaded_TCGA/rna_data.rds Downloaded_TCGA/clinical_data.csv Downloaded_TCGA/methyl_data.rds math_score_data.csv PGI_subtypes.csv

In [ ]:
# process RNA data with cpm threshold of 0.5
!Rscript process_rna.r Downloaded_TCGA/rna_data.rds Downloaded_TCGA/rna_data_processed.rds 0.5

In [ ]:
#four subtypes: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5785562/
#three immune subtypes
gene_sets = ["HALLMARK_ANGIOGENESIS",
             "HALLMARK_APOPTOSIS",
             "HALLMARK_DNA_REPAIR",
             "HALLMARK_E2F_TARGETS",
             "HALLMARK_G2M_CHECKPOINT",
             "HALLMARK_P53_PATHWAY",
             "HALLMARK_KRAS_SIGNALING_DN",
             "HALLMARK_KRAS_SIGNALING_UP",
             "HP_GASTROESOPHAGEAL_REFLUX C5",
             "HP_GASTROPARESIS",
             "HP_ABNORMAL_STOMACH_MORPHOLOGY", 
             "HP_ABSENCE_OF_STOMACH_BUBBLE_ON_FETAL_SONOGRAPHY",
             "HP_STOMACH_CANCER",
             "HP_VOMITING",
             "HP_ABNORMALITY_OF_CHROMOSOME_STABILITY"
             "WOOD_EBV_EBNA1_TARGETS_UP",
             "WP_GASTRIC_CANCER_NETWORK_2",
             "KIM_GASTRIC_CANCER_CHEMOSENSITIVITY",
             "WP_GASTRIC_CANCER_NETWORK_1",
             "VECCHI_GASTRIC_CANCER_ADVANCED_VS_EARLY_UP",
             "WP_CHROMOSOMAL_AND_MICROSATELLITE_INSTABILITY_IN_COLORECTAL_CANCER"
             "VECCHI_GASTRIC_CANCER_EARLY_DN",
             "KOINUMA_COLON_CANCER_MSI_DN",
             "KOINUMA_COLON_CANCER_MSI_UP",
             "KEGG_MISMATCH_REPAIR",
             "GOBP_REGULATION_OF_DNA_METABOLIC_PROCESS"
             "WOOD_EBV_EBNA1_TARGETS_DN",
             "Human Gene Set: SENGUPTA_EBNA1_ANTICORRELATED"
#             "HSIAO_HOUSEKEEPING_GENES" # housekeeping genes
            ]

In [ ]:
open("Downloaded_TCGA/gene_set_selection.txt", "w+").write('\n'.join(gene_sets)+'\n')

In [ ]:
# apply singscore to rna-seq data using rds, gene_selection.txt
!Rscript singscore_test.r Downloaded_TCGA/rna_data_processed.rds Downloaded_TCGA/gene_set_selection.txt H,C5,C2 F logFPKM_TMM Downloaded_TCGA/singscore.csv
# 1: input rnaseq rds fname
# 2: gene set fname with list of selected gene sets relevant to GI cancer
# 3: comma seperated gene set categories (no space) ex: H,C5
# 4: T/F known direciton
# 5: rnaseq metric ex: gene level sum/total TPM
# 6: output csv name/path

In [ ]:
# uses ssgsea R implimentation from" https://rpubs.com/pranali018/SSGSEA
!Rscript ssgsea_test.r Downloaded_TCGA/gene_set_selection.txt H,C5,C2 T T Downloaded_TCGA/rna_data_processed.rds tpm_unstrand Downloaded_TCGA/ssgsea_data.csv
# ssgsea_rpub = function(X, gene_sets, alpha = 0.25, scale = T, norm = T, single = T)
# 1: gene set fname with list of selected gene sets relevant to GI cancer
# 2: comma seperated gene set categories (no space) ex: H,C5
# 3: T/F scaled
# 4: T/F normalized
# 5: rnaseq rds fname
# 6: rnaseq metric ex: gene level sum/total TPM
# 7: output csv name/path

In [ ]:
# leave methylation data for now | SNV data left out of this notebook
# !Rscript process_methyl.r Downloaded_TCGA/methyl_data.rds Downloaded_TCGA/methyl_mvals.csv

In [ ]:
# methyl_df = pd.read_csv("methyl_mvals.csv", index_col=0)
#methyl_df = pd.read_csv("methyl_mvals.csv", index_col=0)
#methyl_selected = methyl_df.loc[["cg25932713", "cg25239996", "cg26556719", "cg02968557"]]
#methyl_selected = methyl_selected.transpose()
#methyl_selected.index = methyl_selected.index.map(lambda x: '-'.join(x.split('-')[:3]))
#methyl_selected = methyl_selected.rename_axis("bcr_patient_barcode")
#methyl_selected.head()

In [ ]:
mathscore_df = pd.read_csv("math_data.csv", index_col=0).rename(columns={"Tumor_Sample_Barcode":"bcr_patient_barcode"})
mathscore_df["bcr_patient_barcode"] = mathscore_df["bcr_patient_barcode"].map(lambda x: '-'.join(x.split('-')[:3]))
mathscore_df = mathscore_df.set_index("bcr_patient_barcode")

clinical_df = pd.read_csv("Downloaded_TCGA/clinical_data.csv", index_col=2)
clinical_df = clinical_df.rename_axis("bcr_patient_barcode")
clinical_df = clinical_df.drop("Unnamed: 0", axis=1)

In [ ]:
ssgsea_df = pd.read_csv("Downloaded_TCGA/ssgsea_data.csv", index_col=0)
ssgsea_df = ssgsea_df.transpose()
ssgsea_df.index = ssgsea_df.index.map(lambda x: '-'.join(x.split('-')[:3]))
new_cols = ssgsea_df.columns.map(lambda x: x+"_ssgsea") 
ssgsea_df = ssgsea_df.rename(columns=dict(zip(list(ssgsea_df.columns),list(new_cols))))
ssgsea_df = ssgsea_df.rename_axis("bcr_patient_barcode")
#ssgsea_df.head()

In [ ]:
#first 30 samples
# plt.figure(figsize=(25, 5))
# sns.heatmap(ssgsea_df.iloc[:, : 30], annot=True)
# plt.show()

In [ ]:
singscore_df = pd.read_csv("Downloaded_TCGA/singscore.csv", index_col=0)
singscore_df.index = singscore_df.index.map(lambda x: '-'.join(x.split('-')[:3]))
singscore_df = singscore_df.rename_axis("bcr_patient_barcode")
#singscore_df.head()

In [ ]:
subtype_df = pd.read_csv("Downloaded_TCGA/PGI_subtypes.csv", index_col=1, encoding='utf-8')
subtype_df = subtype_df.drop("Unnamed: 0", axis=1)
subtype_df = subtype_df.rename_axis("bcr_patient_barcode")
#subtype_df.head()

In [ ]:
#result = pd.concat([clinical_df,mathscore_df, singscore_df, ssgsea_df], axis=1)
singscore_merged = clinical_df.join(mathscore_df)
singscore_merged = singscore_merged.join(methyl_selected)
singscore_merged = singscore_merged.join(singscore_df)

ssgsea_merged = clinical_df.join(mathscore_df)
ssgsea_merged = ssgsea_merged.join(methyl_selected)
ssgsea_merged = ssgsea_merged.join(ssgsea_df)

In [ ]:
#https://docs.gdc.cancer.gov/Data_Dictionary/viewer/#?view=table-definition-view&id=diagnosis&anchor=ajcc_clinical_t
categorical_cols = ["tissue_or_organ_of_origin",
                    "primary_diagnosis",
                    "prior_malignancy",
#                    "ajcc_staging_system_edition",
                    "ajcc_pathologic_t",
                    "morphology",
                    "ajcc_pathologic_n",
                    "ajcc_pathologic_m",
#                    "icd_10_code",
                    "site_of_resection_or_biopsy",
                    "race",
                    "ethnicity",
                    "treatments_radiation_treatment_or_therapy"]

binary_cols = ["prior_malignancy",
               "gender",
               "vital_status"]

numerical_cols = ["age_at_diagnosis",
                   "year_of_diagnosis",
                   "Frame_Shift_Del",
                   "Frame_Shift_Ins",
                   "In_Frame_Del",
                   "In_Frame_Ins",
                   "Missense_Mutation",
                   "Nonsense_Mutation",
                   "Nonstop_Mutation",
                   "Splice_Site",
                   "Translation_Start_Site",
                   "MedianAbsoluteDeviation",
                   "MATH"]


In [ ]:
singscore_cols = ["TotalScore_%s" % i for i in gene_sets if "TotalScore_%s" % i in singscore_df.columns]
singscore_cols += ["TotalDispersion_%s" % i for i in gene_sets if "TotalDispersion_%s" % i in singscore_df.columns]
singscore_cols += numerical_cols+binary_cols+categorical_cols
singscore_cols_meth = singscore_cols +["cg25932713", "cg25239996", "cg26556719", "cg02968557"]

In [ ]:
ssgsea_cols = ["%s_ssgsea" % i for i in gene_sets if "%s_ssgsea" % i in ssgsea_df.columns]
ssgsea_cols += numerical_cols+binary_cols+categorical_cols
ssgsea_cols_meth = ssgsea_cols+["cg25932713", "cg25239996", "cg26556719", "cg02968557"]

In [ ]:
singscore_selected_df = singscore_merged[singscore_cols].dropna()
singscore_selected_meth_df = singscore_merged[singscore_cols_meth].dropna()

ssgsea_selected_df = ssgsea_merged[ssgsea_cols].dropna()
ssgsea_selected_meth_df = ssgsea_merged[ssgsea_cols_meth].dropna()

In [ ]:
print(len(singscore_selected_df.index))
#singscore_selected_df.head()

In [ ]:
print(len(ssgsea_selected_df.index))
#ssgsea_selected_df.head()

In [ ]:
singscore_selected_df.to_csv("Downloaded_TCGA/singscore_selected_df.csv")
ssgsea_selected_df.to_csv("Downloaded_TCGA/ssgsea_selected_df.csv")
#singscore_selected_meth_df.to_csv("Downloaded_TCGA/singscore_selected_meth_df.csv")
#ssgsea_selected_meth_df.to_csv("Downloaded_TCGA/ssgsea_selected_meth_df.csv")

In [ ]:
input_data_fname = "Downloaded_TCGA/ssgsea_selected_df.csv"
X = pd.read_csv(input_data_fname, index_col=[0], header=[0])
X_original = X
feature_list = X.columns.to_list()
#X.head()

In [ ]:
{ix:col for ix,col in enumerate(feature_list)}

In [ ]:
from sparsemedoid import clustering
from sklearn import metrics

In [ ]:
clusters = [2,3,4,5]
distance_types = ["gower", "wishart", "podani"]
normalization_param = [1.01, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0]

total_runs = (len(clusters) * len(distance_types) * len(normalization_param))
Scores = np.zeros((1, total_runs))
X_df = pd.read_csv(input_data_fname, index_col=[0], header=[0])
X = X_df[to_keep].to_numpy()
P = X.shape[0]
N = X.shape[1]
prefix_cols = []

all_feature_weights = np.zeros((N, total_runs))
all_cluster_labels = np.zeros((P, total_runs))
iter1 = 0

tic()
for K in clusters:
    for distance in distance_types:
        for S in normalization_param:
            results_path_prefix = f"K={K}_dist={distance}_S={S}"
            #result_labels.append(results_path_prefix)
            prefix_col = f"N={N}_K={K}_dist={distance}_nparam={S}"
            prefix_cols.append(results_path_prefix)
            (
                cluster_labels,
                feature_weights,
                feature_order,
                weighted_distances,
            ) = clustering.sparse_kmedoids(
                X,
                distance_type=distance,
                k=K,
                s=S,
                max_attempts=6,
                method="pam",
                init="build",
                max_iter=100,
                random_state=None,
            )
            Scores[0, iter1] += metrics.silhouette_score(
                weighted_distances, cluster_labels, metric="precomputed"
            )
            all_feature_weights[:, iter1] = feature_weights
            all_cluster_labels[:, iter1] = cluster_labels

            iter1 += 1
            # weighted_distances_df = pd.DataFrame(weighted_distances)
toc()
feature_weights_df = pd.DataFrame(all_feature_weights)
cluster_labels_df = pd.DataFrame(all_cluster_labels)
scores_df = pd.DataFrame(Scores)

In [ ]:
rename_rows = {ix: to_keep[ix] for ix in feature_weights_df.index}
rename_cols = {ix: prefix_cols[ix] for ix in feature_weights_df.columns}

fw_df = feature_weights_df.rename(index=rename_rows)
fw_df = fw_df.rename(columns=rename_cols)

In [ ]:
cluster_labels_df.index = X_original.index
cl_df = cluster_labels_df.rename(columns=rename_cols)

In [ ]:
label_df = subtype_df[subtype_df["cancer.type"]=="STAD"][["Subtype_Selected"]]
label_list = label_df.index

In [ ]:
cl_w_label = cl_df.join(label_df)
#cl_w_label.head()

In [ ]:
GI_types = [i for i in list(set(cl_w_label["Subtype_Selected"])) if "GI" in str(i)]
GI_types

In [ ]:
def format_valid_output(dist_type, k, s):
    run_key = "K=%i_dist=%s_S=%s" %(int(k), dist_type, str(s))
    number_types_dict={}
    for typei in GI_types:
        number_types_dict[typei] = []
        gi_subset = cl_w_label[cl_w_label["Subtype_Selected"]==typei]
        #number_types_dict[typei] = len(gi_subset.index)
        for i in range(0,clusters[-1]):
            cluster_sub = gi_subset[gi_subset[run_key] == i]
            number_types_dict[typei].append(len(cluster_sub.index))
    df = pd.DataFrame(columns=range(0,5)).from_dict(number_types_dict)
    df = df.rename_axis("cluster label")
    return df

In [ ]:
#distance_types = ["gower", "wishart", "podani"]
#normalization_param = [1.01, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0] 
#clusters = [2,3,4,5]

for k in clusters:
    f, ax = plt.subplots(len(normalization_param), len(distance_types), sharey=True)
    for j, dist_type in enumerate(distance_types):
        for i, s in enumerate(normalization_param):
            run_key = "K=%i_dist=%s_S=%s" %(int(k), dist_type, str(s))
            format_valid_output(dist_type, k, s).transpose().plot(kind = 'barh', 
                                                              stacked = True, 
                                                              title = 'Stacked Bar Graph', 
                                                              mark_right = True,
                                                              ax=ax[i][j]).set_title(run_key)
    f.set_figwidth(20)
    f.set_figheight(20)
    plt.tight_layout()
    plt.show()

In [ ]:
scores_df = scores_df.rename(columns=rename_cols)
fig, ax = plt.subplots(figsize=(20, 5))
sns.barplot(scores_df, ax=ax)
plt.xticks(rotation=90)
plt.show()

In [ ]:
labels = pd.DataFrame(cl_df.nunique(axis=0), columns=["nunique"]).T
fig, ax = plt.subplots(figsize=(20, 5))
sns.barplot(labels, ax=ax)
plt.xticks(rotation=90)
plt.show()

In [ ]:
import plotly.express as px

def get_weights(cluster_num, distance_type):
    cl_token = "K=%i" % cluster_num
    dist_token = "dist=%s" % distance_type
    run_title = "Weights-%s-%s" % (cl_token, dist_token)
    col_accept = [col for col in fw_df.columns if dist_token in col and cl_token in col]
    transf_fw = fw_df[col_accept].T
    S_vals = [float(run_name.split("_S=")[-1]) for run_name in transf_fw.index]
    transf_fw["S"] = S_vals
    feature_data = {feature:transf_fw[["S",feature]].reset_index(drop=True) for feature in to_keep}
    all_df = []
    for f_ix in feature_data:
        f_df = feature_data[f_ix]
        f_df["feature"] = [f_ix]*len(f_df)
        f_df = f_df.rename(columns={f_ix:"Weight"})
        all_df.append(f_df)
    merged_weights = pd.concat(all_df, axis=0, ignore_index=True)
    fig = px.line(merged_weights, x='S', y='Weight', color="feature", title=run_title)
    fig.update_traces(mode="markers+lines")
    fig.write_html("%s.html" % run_title)


def get_labels(cluster_num, distance_type):
    cl_token = "K=%i" % cluster_num
    dist_token = "dist=%s" % distance_type
    col_accept = [col for col in fw_df.columns if dist_token in col and cl_token in col]
    run_title = "labels-%s-%s" % (cl_token, dist_token)
    transf_cl = cl_df[col_accept]
    cl_data = {}
    for col in col_accept:
        cl_data[col] = []
        for label in range(0,cluster_num):
            perc_label = list(transf_cl[col]).count(label)/len(transf_cl[col])
            cl_data[col].append(perc_label)
    new_cl_df = pd.DataFrame().from_dict(cl_data).T
    S_vals = [float(run_name.split("_S=")[-1]) for run_name in new_cl_df.index]
    new_cl_df["S"] = S_vals
    plt.figure(figsize=(10,5))
    for label in range(0,cluster_num):
        sns.lineplot(x="S",
                    y=label,
                    data=new_cl_df,
                    label=label)
    
    plt.ylabel('prop. label')
    #plt.show()
    #return new_cl_df.head()


In [ ]:
#get_weights(5, "gower")
#get_labels(4, "gower")
#get_labels(4, "podani")

#### Links

1. https://www.bioconductor.org/packages/release/workflows/vignettes/TCGAWorkflow/inst/doc/TCGAWorkflow.html
2. https://github.com/hamidghaedi/Methylation_Analysis/blob/master/cross_reactive_probe.chen2013.csv
3. https://github.com/hamidghaedi/Methylation_Analysis
4. https://nbis-workshop-epigenomics.readthedocs.io/en/latest/content/tutorials/methylationArray/Array_Tutorial.html
5. https://life-epigenetics-methylprep.readthedocs-hosted.com
6. https://bioconductor.org/packages/devel/bioc/vignettes/singscore/inst/doc/singscore.html
7. https://cran.r-project.org/web/packages/msigdbr/vignettes/msigdbr-intro.html
8. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6844140/

In [ ]:
#sns.set(style='whitegrid')
#sns.scatterplot(x="Hallmark_Vomitting_Singscore",
#                    y="Hallmark_Vomitting_Dispersion",
#                    data=X_original)

#sns.scatterplot(x="EBV_Viremia_Singscore.1",
#                y="EBV_Viremia_Dispersion.1",
#                data=X_original)  
#sns.scatterplot(x="GI_Hallmark_Singscore",
#                y="GI_Hallmark_Dispersion",
#                hue="tissue_or_organ_of_origin",
#                data=X_original)    
#sns.scatterplot(x="Hallmark_PI3K_Pathway_Singscore",
#                y="Hallmark_PI3K_Pathway_Dispersion",
#                hue="MATH",
#                data=X_original)  

#sns.scatterplot(x="HTLV1_Infection_Singscore",
#                y="HTLV1_Infection_Dispersion",
#                hue="gender",
#                data=X_original) 